In [1]:
import tensorflow as tf

In [2]:
print(tf.__version__)
tf.config.list_physical_devices('GPU')

2.5.0


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [5]:
path = '/gdrive/My Drive/'

<h1> Load Module </h1>

In [6]:
from keras import models, layers, optimizers, utils
from keras.layers import Dropout, Activation, Dense, BatchNormalization
from keras.models import load_model

In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import keras
from keras.preprocessing.image import load_img
import random
import cv2, os, sys
from keras.preprocessing import image
from PIL import Image

In [8]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array
from numpy import expand_dims

<h1> Load Image </h1>

In [9]:
# 이미지 크기 지정

IMAGE_WIDTH = 224
IMAGE_HEIGHT = 224

In [10]:
# 트레이닝 데이터 파일을 정렬
import natsort
train_order = os.listdir(path+'04_multimodal_training')
train_order = natsort.natsorted(train_order)

In [12]:
# Image Augmentation 생성
datagen = ImageDataGenerator(
            width_shift_range = 0.01,
            height_shift_range = 0.01,
            horizontal_flip = True,
            vertical_flip = True,
            zoom_range = 0.05,
            brightness_range=[1.0, 1.0])

In [13]:
# 이미지를 64개의 카테고리로 분류
nb_classes = 64

data = []
label= []
dtype = []

for filename in train_order:
    category = filename[:3]
    imagePath = path + '/04_multimodal_training/' + filename
    if filename[4:8] == 'face':
      dtype.append('F')
    elif filename[4:8] == 'iris':
      dtype.append('I')

    image = cv2.imread(imagePath)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (IMAGE_WIDTH, IMAGE_HEIGHT))
    
    data.append(image)
    label.append(int(category))
    new_image = img_to_array(image)
    samples = expand_dims(new_image, 0)
    it= datagen.flow(samples, batch_size=1)

    # 이미지를 5개씩 augmentation
    for i in range(5):
        batch = it.next()
        image = batch[0].astype('uint8')
        np_image = np.array(image)

        data.append(np_image)
        label.append(int(category))
        
        # 이미지의 종류에 따라 타입 설정
        if filename[4:8] == 'face':
          dtype.append('F')
        elif filename[4:8] == 'iris':
          dtype.append('I')

In [14]:
# iris와 face 각각 훈련데이터셋 형성

face_X = []
face_Y = []
iris_X = []
iris_Y = []

for i in range((len(data))):
  if dtype[i] == 'F':
    face_X.append(data[i])
    face_Y.append(label[i])
  elif dtype[i] == 'I':
    iris_X.append(data[i])
    iris_Y.append(label[i])

<h1> Build Multimodal Model </h1>

In [15]:
from keras import Sequential
from keras import models, layers, optimizers, utils
from keras.layers import Dropout, Activation, Dense
from keras.layers import Flatten, Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.models import load_model

In [16]:
# CNN 모델 생성

def build_model():

    learning_rate = 0.0001
    model = Sequential()
    model.add(ZeroPadding2D((1,1),input_shape=(IMAGE_HEIGHT, IMAGE_WIDTH, 3)))
    model.add(Convolution2D(64, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))
    
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))
    
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))
    
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))
    
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))
    
    model.add(Convolution2D(4096, (7, 7), activation='relu'))
    model.add(Dropout(0.5))
    model.add(Convolution2D(4096, (1, 1), activation='relu'))
    model.add(Dropout(0.5))
    model.add(Convolution2D(2622, (1, 1)))
    model.add(Flatten())
    model.add(Activation('softmax'))
    
    return model

In [17]:
# VGG face 가중치 사용

model = build_model()
model.load_weights(path+'/vgg_face_weights.h5')
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding2d (ZeroPadding2 (None, 226, 226, 3)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 224, 224, 64)      1792      
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 226, 226, 64)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 224, 224, 64)      36928     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 112, 112, 64)      0         
_________________________________________________________________
zero_padding2d_2 (ZeroPaddin (None, 114, 114, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 112, 112, 128)     7

In [18]:
from tensorflow.keras.models import Model
descriptor = Model(inputs=model.layers[0].input, outputs=model.layers[-2].output)

In [19]:
face_X = np.array(face_X)
iris_X = np.array(iris_X)
face_Y = np.array(face_Y)
iris_Y = np.array(iris_Y)

In [20]:
# 이미지를 64개의 카테고리로 분류
nb_classes = 64

face_embeddings = np.zeros((nb_classes*24, 2622))

i = 0

for image in face_X:
    image = (image / 255.).astype(np.float32)
    image = cv2.resize(image, (IMAGE_WIDTH, IMAGE_HEIGHT), cv2.INTER_AREA)
    # descriptor 모델에서 나온 특징점들을 저장
    face_embeddings[i]= descriptor.predict(np.expand_dims(image, axis=0))[0]
    i += 1

In [21]:
# 이미지를 64개의 카테고리로 분류
nb_classes = 64

iris_embeddings = np.zeros((nb_classes*24, 2622))

i = 0

for image in iris_X:
    image = (image / 255.).astype(np.float32)
    image = cv2.resize(image, (IMAGE_WIDTH, IMAGE_HEIGHT), cv2.INTER_AREA)
    # face_descriptor 모델에서 나온 특징점들을 저장
    iris_embeddings[i]= descriptor.predict(np.expand_dims(image, axis=0))[0]
    i += 1

In [22]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

In [23]:
# CNN으로 예측한 홍채와 얼굴 특징점 데이터를 하나의 특징으로 합침
embeddings = []

i = 0
for i in range(len(iris_embeddings)):
  embeddings.append(face_embeddings[i] + iris_embeddings[i])
  i += 1

In [24]:
embeddings = np.array(embeddings)

In [25]:
# 특징 데이터들을 train과 test로 분할
# 4개의 데이터마다 하나씩 테스트 데이터로 사용
train_idx = np.arange(nb_classes*24) % 4 != 0
test_idx = np.arange(nb_classes*24) % 4 == 0

X_train = embeddings[train_idx]
X_test = embeddings[test_idx]
Y_train = face_Y[train_idx]
Y_test = face_Y[test_idx]

In [26]:
# y_train과 y_test 값을 라벨링
encoder = LabelEncoder()
Y_train = encoder.fit_transform(Y_train)
Y_test = encoder.transform(Y_test)

In [27]:
# SVM으로 학습 데이터 훈련
svc = LinearSVC(C = 10) 
svc.fit(X_train, Y_train)
y_pred = svc.predict(X_train)

# 3번의 cross validation을 통해 training set score 교차 검증
print("Accuracy on training set: ", cross_val_score(svc, X_train, Y_train, cv=3))
print("Accuracy on test set: ", svc.score(X_test, Y_test))

Accuracy on training set:  [1.         0.99739583 0.9609375 ]
Accuracy on test set:  1.0


In [28]:
# model accuracy

y_test_pred = svc.predict(X_test)

print("accuracy: {:.2f}".format(accuracy_score(Y_test, y_test_pred)))
print("Precision: {:.2f}".format(precision_score(Y_test, y_test_pred, average = 'micro')))
print("Recall: {:.2f}".format(recall_score(Y_test, y_test_pred, average = 'micro')))
print("F1-score: {:.2f}".format(f1_score(Y_test, y_test_pred, average = 'micro')))

accuracy: 1.00
Precision: 1.00
Recall: 1.00
F1-score: 1.00


<h1> Test </h1>

In [29]:
import pandas as pd

In [30]:
# 테스트데이터 파일을 숫자 순으로 정렬
import natsort
order_list = os.listdir(path+'04_multimodal_test')
order_list = natsort.natsorted(order_list)

In [31]:
print(order_list)

['0_face.png', '0_iris.png', '1_face.png', '1_iris.png', '2_face.png', '2_iris.png', '3_face.png', '3_iris.png', '4_face.png', '4_iris.png', '5_face.png', '5_iris.png', '6_face.png', '6_iris.png', '7_face.png', '7_iris.png', '8_face.png', '8_iris.png', '9_face.png', '9_iris.png', '10_face.png', '10_iris.png', '11_face.png', '11_iris.png', '12_face.png', '12_iris.png', '13_face.png', '13_iris.png', '14_face.png', '14_iris.png', '15_face.png', '15_iris.png', '16_face.png', '16_iris.png', '17_face.png', '17_iris.png', '18_face.png', '18_iris.png', '19_face.png', '19_iris.png', '20_face.png', '20_iris.png', '21_face.png', '21_iris.png', '22_face.png', '22_iris.png', '23_face.png', '23_iris.png', '24_face.png', '24_iris.png', '25_face.png', '25_iris.png', '26_face.png', '26_iris.png', '27_face.png', '27_iris.png', '28_face.png', '28_iris.png', '29_face.png', '29_iris.png', '30_face.png', '30_iris.png', '31_face.png', '31_iris.png', '32_face.png', '32_iris.png', '33_face.png', '33_iris.png',

In [ ]:
# face와 test 각각 배열에 저장
face_test = []
iris_test = []

for filename in order_list:
    imagePath = path + '/04_multimodal_test/' + filename

    image = cv2.imread(imagePath)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (IMAGE_WIDTH, IMAGE_HEIGHT))

    new_image = (image / 255.).astype(np.float32)
    new_image = cv2.resize(new_image, (IMAGE_WIDTH, IMAGE_HEIGHT), cv2.INTER_AREA)

    if filename[-8:-4] == 'face':
      face_test.append(face_descriptor.predict(np.expand_dims(new_image, axis=0))[0])
    elif filename[-8:-4] == 'iris':
      iris_test.append(face_descriptor.predict(np.expand_dims(new_image, axis=0))[0])

In [ ]:
face_test = np.array(face_test)
iris_test = np.array(iris_test)

In [ ]:
test_embeddings = []

i = 0
for i in range(len(face_test)):
  test_embeddings.append(face_test[i] + iris_test[i])
  i += 1

In [ ]:
# face model predict
predict = svc.predict(test_embeddings)

In [ ]:
# face model result
predict

array([27, 31, 22, 57, 54, 45,  5, 12,  8,  7, 36,  2, 46, 50, 10, 61, 49,
       59, 56, 13, 37, 30, 25,  0, 41, 58,  9, 20, 48, 47, 42, 24,  3,  6,
       55, 63, 21, 16, 34, 11, 35, 32, 40, 62, 19, 52, 33, 15,  1, 29, 17,
       26, 53, 28, 18, 14, 43, 60,  4, 39, 38, 23, 51, 44])

<h1> Result </h1>

In [ ]:
result = pd.DataFrame({'Image':[],
                 'Answer':[]})

In [ ]:
for i in range(len(predict)):
  new_data = {'Image':int(i), 'Answer':int(predict[i])}
  result = result.append(new_data, ignore_index = True)

In [ ]:
result

,Image,Answer
0,0.0,27.0
1,1.0,31.0
2,2.0,22.0
3,3.0,57.0
4,4.0,54.0
...,...,...
59,59.0,39.0
60,60.0,38.0
61,61.0,23.0
62,62.0,51.0


In [ ]:
result.to_csv('/gdrive/My Drive/multimodal_result2.csv')